<a href="https://colab.research.google.com/github/kanri3/deep_learning_day1_day2/blob/main/1_2_back_propagation_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importと関数定義

In [ ]:
import numpy as np
# from common import functions
# import matplotlib.pyplot as plt

def print_vec(text, vec):
    print("*** " + text + " ***")
    np.set_printoptions(precision=5)
    print(vec)
    print("shape:", vec.shape)
    #print("shape: " + str(x.shape))
    # print("")


# ReLU関数
def relu(x):
    return np.maximum(0, x)
# 中間層の活性化関数
# シグモイド関数（ロジスティック関数）
def sigmoid(x):
    return 1/(1 + np.exp(-x))
# 出力層の活性化関数
# ソフトマックス関数
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T
    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))
# クロスエントロピー
def cross_entropy_error(d, y):
    if y.ndim == 1:
        d = d.reshape(1, d.size)
        y = y.reshape(1, y.size)
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    # One-hotベクトルとは、(0,1,0,0,0,0) の様に、1つの成分が1で残りの成分が全て0であるベクトルのこと
    if d.size == y.size:
        # NumPyのargmax関数は、多次元配列の中の最大値の要素を持つインデックスを返す関数
        d = d.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), d] + 1e-7)) / batch_size
# 誤差関数
# 平均二乗誤差
def mean_squared_error(d, y):
    return np.mean(np.square(d - y)) / 2
# シグモイドとクロスエントロピーの複合導関数
def d_sigmoid_with_loss(d, y):
    return y - d
# ReLU関数の導関数
def d_relu(x):
    return np.where( x > 0, 1, 0)

# メインプログラム

In [ ]:
# ウェイトとバイアスを設定
# ネートワークを作成
def init_network():
    print("##### ネットワークの初期化 #####")
    network = {}
    network['W1'] = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6]
    ])
    network['W2'] = np.array([
        [0.1, 0.4],
        [0.2, 0.5],
        [0.3, 0.6]
    ])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['b2'] = np.array([0.1, 0.2])
    print_vec("重み1", network['W1'])
    print_vec("重み2", network['W2'])
    print_vec("バイアス1", network['b1'])
    print_vec("バイアス2", network['b2'])

    return network

# 順伝播
def forward(network, x):
    print("##### 順伝播開始 #####")
    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']
    
    u1 = x @ W1 + b1
    # u1 = np.dot(x, W1) + b1
    z1 = relu(u1)
    # z1 = functions.relu(u1)
    u2 = z1 @ W2 + b2
    # u2 = np.dot(z1, W2) + b2
    y = softmax(u2)
    # y = functions.softmax(u2)
    
    print_vec("総入力1", u1)
    print_vec("中間層出力1", z1)
    print_vec("総入力2", u2)
    print_vec("出力1", y)
    print("出力合計: " + str(np.sum(y)))
    return y, z1

# 誤差逆伝播
def backward(x, d, z1, y):
    print("\n##### 誤差逆伝播開始 #####")
    grad = {}
    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']
    #  出力層でのデルタ
    delta2 = y - d
    # delta2 = d_sigmoid_with_loss(d, y)
    # delta2 = functions.d_sigmoid_with_loss(d, y)
    #  b2の勾配
    grad['b2'] = np.sum(delta2, axis=0)
    #  W2の勾配
    grad['W2'] = z1.T @ delta2
    # grad['W2'] = np.dot(z1.T, delta2)
    #  中間層でのデルタ
    delta1 = (delta2 @ W2.T) * d_relu(z1)
    # delta1 = np.dot(delta2, W2.T) * functions.d_relu(z1)
    # b1の勾配
    grad['b1'] = np.sum(delta1, axis=0)
    #  W1の勾配
    grad['W1'] = x.T @ delta1
    # grad['W1'] = np.dot(x.T, delta1)
        
    print_vec("偏微分_dE/du2", delta2)
    print_vec("偏微分_dE/du2", delta1)
    print_vec("偏微分_重み1", grad["W1"])
    print_vec("偏微分_重み2", grad["W2"])
    print_vec("偏微分_バイアス1", grad["b1"])
    print_vec("偏微分_バイアス2", grad["b2"])
    return grad
    
# 訓練データ
x = np.array([[1.0, 5.0]])
# 目標出力
d = np.array([[0, 1]])
#  学習率
learning_rate = 0.01
network =  init_network()
y, z1 = forward(network, x)

# 誤差
# loss = cross_entropy_error(d, y)
# loss = functions.cross_entropy_error(d, y)

grad = backward(x, d, z1, y)
num=0
for key in ('W1', 'W2', 'b1', 'b2'):
    network[key]  -= learning_rate * grad[key]
    print("num:", num)
    num+=1

print("##### 結果表示 #####")    
print("##### 更新後パラメータ #####") 
print_vec("重み1", network['W1'])
print_vec("重み2", network['W2'])
print_vec("バイアス1", network['b1'])
print_vec("バイアス2", network['b2'])

##### ネットワークの初期化 #####
*** 重み1 ***
[[0.1 0.3 0.5]
 [0.2 0.4 0.6]]
shape: (2, 3)
*** 重み2 ***
[[0.1 0.4]
 [0.2 0.5]
 [0.3 0.6]]
shape: (3, 2)
*** バイアス1 ***
[0.1 0.2 0.3]
shape: (3,)
*** バイアス2 ***
[0.1 0.2]
shape: (2,)
##### 順伝播開始 #####
*** 総入力1 ***
[[1.2 2.5 3.8]]
shape: (1, 3)
*** 中間層出力1 ***
[[1.2 2.5 3.8]]
shape: (1, 3)
*** 総入力2 ***
[[1.86 4.21]]
shape: (1, 2)
*** 出力1 ***
[[0.08707 0.91293]]
shape: (1, 2)
出力合計: 1.0

##### 誤差逆伝播開始 #####
*** 偏微分_dE/du2 ***
[[ 0.08707 -0.08707]]
shape: (1, 2)
*** 偏微分_dE/du2 ***
[[-0.02612 -0.02612 -0.02612]]
shape: (1, 3)
*** 偏微分_重み1 ***
[[-0.02612 -0.02612 -0.02612]
 [-0.1306  -0.1306  -0.1306 ]]
shape: (2, 3)
*** 偏微分_重み2 ***
[[ 0.10448 -0.10448]
 [ 0.21766 -0.21766]
 [ 0.33085 -0.33085]]
shape: (3, 2)
*** 偏微分_バイアス1 ***
[-0.02612 -0.02612 -0.02612]
shape: (3,)
*** 偏微分_バイアス2 ***
[ 0.08707 -0.08707]
shape: (2,)
num: 0
num: 1
num: 2
num: 3
##### 結果表示 #####
##### 更新後パラメータ #####
*** 重み1 ***
[[0.10026 0.30026 0.50026]
 [0.20131 0.40131 0.60131]]
shape: (2, 3)
